In [39]:
import scipy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pydicom
import cv2
from skimage import filters

In [40]:
def histogram_equalization(img):
    m = int(np.max(img))
    hist = np.histogram(img, bins=m+1, range=(0, m+1))[0]
    hist = hist/img.size
    cdf = np.cumsum(hist)
    s_k = (255 * cdf)
    img_new = np.array([s_k[i] for i in img.ravel()]).reshape(img.shape)
    return img_new

In [41]:
def get_masks_and_sizes_of_connected_components(img_mask):
    """
    Finds the connected components from the mask of the image
    """
    mask, num_labels = scipy.ndimage.label(img_mask)

    mask_pixels_dict = {}
    for i in range(num_labels+1):
        this_mask = (mask == i)
        if img_mask[this_mask][0] != 0:
            # Exclude the 0-valued mask
            mask_pixels_dict[i] = np.sum(this_mask)
    return mask, mask_pixels_dict

In [42]:
def get_mask_of_largest_connected_component(img_mask):
    """
    Finds the largest connected component from the mask of the image
    """
    mask, mask_pixels_dict = get_masks_and_sizes_of_connected_components(img_mask)
    largest_mask_index = pd.Series(mask_pixels_dict).idxmax()
    largest_mask = mask == largest_mask_index
    return largest_mask

In [43]:
def image_processing(img):
    threshold = filters.threshold_isodata(img)
    bin_img = (img > threshold)*1
    kernel = np.ones((5, 5), np.uint8)
    bin_img = bin_img.astype('uint8')
    bin_img = cv2.erode(bin_img, kernel, iterations=-2)
    
    img_mask = get_mask_of_largest_connected_component(bin_img)
    
    farthest_pixel = np.max(list(zip(*np.where(img_mask == 1))), axis=0)
    nearest_pixel = np.min(list(zip(*np.where(img_mask == 1))), axis=0)
    cropped =  img[nearest_pixel[0]:farthest_pixel[0], nearest_pixel[1]:farthest_pixel[1]]
    return cropped

In [44]:
%%time

in_path = './images/10130/2110820077.dcm'
ds = pydicom.dcmread(in_path) #Read .dcm file
img = np.array(ds.pixel_array, dtype=np.uint8)

CPU times: total: 2.06 s
Wall time: 2.27 s


In [45]:
%%time

new_img = image_processing(img)

CPU times: total: 23.6 s
Wall time: 24.2 s


In [46]:
%%time

eq_img = histogram_equalization(new_img)

CPU times: total: 15.6 ms
Wall time: 9.54 ms
